# Pre-Compilation

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

size = 14
legendsize = 12

In [ ]:
import math
def orderOfMagnitude(number):
    return -math.ceil(math.log(number, 10))

# MaxCut

In [ ]:
df_maxcut = pd.read_csv('res_qaoa.csv', sep=',')
df_maxcut['num_qubits'] = df_maxcut['num_qubits'].astype(int)
df_maxcut

In [ ]:
def label_encoding (row):
    if row['considered_following_qubits'] == 1 :
        return 'Only Direct Neighbor'
    elif row['considered_following_qubits'] == 1000 :
        return "All Neighbors"
df_maxcut["Encoding Prediction"] = df_maxcut.apply(lambda row: label_encoding(row), axis=1)

## Graph

In [ ]:
for considered_following_qubits in ["Only Direct Neighbor", "All Neighbors"]:
    for sample_probability in [0.3,0.7]:
        df_subset = df_maxcut[(df_maxcut.sample_probability==sample_probability) & (df_maxcut["Encoding Prediction"]==considered_following_qubits)]
        
        ax1 = df_subset.plot(x='num_qubits', y='cx_count_proposed', color='orange', style="o-", label="Proposed Scheme")   
        ax1.tick_params(which='both', labelsize=size)
        df_subset.plot(x='num_qubits', y='cx_count_baseline_O0', color='red', style="x-.", ax=ax1, label="Qiskit's O0")
        df_subset.plot(x='num_qubits', y='cx_count_baseline_O1', color='purple', ax=ax1, style=".--", label="Qiskit's O1") 
        df_subset.plot(x='num_qubits', y='cx_count_baseline_O2', color='blue', ax=ax1, style="+-.", label="Qiskit's O2") 
        df_subset.plot(x='num_qubits', y='cx_count_baseline_O3', color='green', ax=ax1, style="^-.", label="Qiskit's O3") 
        
        plt.xlabel("Qubits", size=size)
        plt.ylabel("Number of two-qubit gates", size=size)
        plt.yscale("log")
        plt.legend(fontsize=legendsize)
        plt.savefig('cx_'+str(considered_following_qubits) + '_'+ str(sample_probability)+'.pdf', bbox_inches="tight")
        plt.show()
        
        ax2 = df_subset.plot(x='num_qubits', y='time_proposed', color='orange', style="o-", label="Proposed Scheme")   
        ax2.tick_params(which='both', labelsize=size)
        df_subset.plot(x='num_qubits', y='time_baseline_O0', color='red', ax=ax2, style="x-.", label="Qiskit's O0")
        df_subset.plot(x='num_qubits', y='time_baseline_O1', color='purple', ax=ax2, style=".--", label="Qiskit's O1") 
        df_subset.plot(x='num_qubits', y='time_baseline_O2', color='blue', ax=ax2, style="+-.", label="Qiskit's O2") 
        df_subset.plot(x='num_qubits', y='time_baseline_O3', color='green', ax=ax2, style="^-.", label="Qiskit's O3") 
        
        plt.xlabel("Qubits", size=size)
        plt.ylabel("Time", size=size)
        plt.yscale("log")
        plt.legend(fontsize=legendsize)
        plt.savefig('time_'+str(considered_following_qubits) + '_'+ str(sample_probability)+'.pdf', bbox_inches="tight")
        plt.show()

## Averages

In [ ]:
df_maxcut["time_ratio_O3"] = df_maxcut["time_proposed"]/df_maxcut["time_baseline_O3"]
df_maxcut["order_magnitudes_diff"] = df_maxcut["time_ratio_O3"].apply(orderOfMagnitude)
df_maxcut["order_magnitudes_diff"].describe()

In [ ]:
df_maxcut["cx_ratio_O3"] = df_maxcut['cx_count_proposed']/df_maxcut['cx_count_baseline_O3']
df_maxcut.cx_ratio_O3.describe()

# Satellite

In [ ]:
df_satellite = pd.read_csv('res_satellite.csv', sep=',')
df_satellite['num_qubits'] = df_satellite['num_qubits'].astype(int)
df_satellite

## Graphs

In [ ]:
df_subset = df_satellite[(df_satellite.sample_probability==0.4) & (df_satellite.considered_following_qubits==1)]

ax1 = df_subset.plot(x='num_qubits', y='cx_count_proposed', color='orange', style="o-", label="Proposed Scheme")   
ax1.tick_params(which='both', labelsize=size)
df_subset.plot(x='num_qubits', y='cx_count_baseline_O0', color='red', style="x-.", ax=ax1, label="Qiskit's O0")
df_subset.plot(x='num_qubits', y='cx_count_baseline_O1', color='purple', ax=ax1, style=".--", label="Qiskit's O1") 
df_subset.plot(x='num_qubits', y='cx_count_baseline_O2', color='blue', ax=ax1, style="+-.", label="Qiskit's O2") 
df_subset.plot(x='num_qubits', y='cx_count_baseline_O3', color='green', ax=ax1, style="^-.", label="Qiskit's O3") 
        

plt.ylim(10e0*1.5, 10e2)
plt.xlabel("Qubits", size=size)
plt.ylabel("Number of two-qubit gates", size=size)
plt.yscale("log")
plt.legend(fontsize=legendsize)
plt.savefig('sat_cx.pdf', bbox_inches="tight")
plt.show()

ax2 = df_subset.plot(x='num_qubits', y='time_proposed', color='orange', style="o-", label="Proposed Scheme")   
ax2.tick_params(which='both', labelsize=size)
df_subset.plot(x='num_qubits', y='time_baseline_O0', color='red', ax=ax2, style="x-.", label="Qiskit's O0")
df_subset.plot(x='num_qubits', y='time_baseline_O1', color='purple', ax=ax2, style=".--", label="Qiskit's O1") 
df_subset.plot(x='num_qubits', y='time_baseline_O2', color='blue', ax=ax2, style="+-.", label="Qiskit's O2") 
df_subset.plot(x='num_qubits', y='time_baseline_O3', color='green', ax=ax2, style="^-.", label="Qiskit's O3") 
        

plt.xlabel("Qubits", size=size)
plt.ylabel("Time", size=size)

plt.yscale("log")
plt.legend(fontsize=legendsize, loc="center right")
plt.savefig('sat_time.pdf', bbox_inches="tight")
plt.show()

In [ ]:
df_satellite["time_ratio_O3"] = df_satellite["time_proposed"]/df_satellite["time_baseline_O3"]
df_satellite["order_magnitudes_diff"] = df_satellite["time_ratio_O3"].apply(orderOfMagnitude)
df_satellite["order_magnitudes_diff"].describe()

In [ ]:
df_satellite["cx_ratio_O3"] = df_satellite["cx_count_proposed"]/df_satellite["cx_count_baseline_O3"]
df_satellite.cx_ratio_O3.describe()